<a href="https://colab.research.google.com/github/NicoloAvesani/Mining_Project/blob/main/that's_good_OK_APRIORI%2C_FP_GROWTH_and_ECLAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dataset generator
https://github.com/jeffheaton/papers/blob/master/2016/ieee-freq-item/GenerateFreqData.py


In [289]:
# This Python script was used to collect the data for following paper/conference:
#
# Heaton, J. (2016, April). Comparing Dataset Characteristics that Favor the Apriori, 
# Eclat or FP-Growth Frequent Itemset Mining Algorithms. In SoutheastCon 2015 (pp. 1-6). IEEE.
#
# http://www.jeffheaton.com
#

# Generate benchmark data for frequent itemset mining.
__author__ = 'jheaton'
import random
import csv
from tqdm import tqdm

def sizeof_fmt(num):
    for x in ['','k','m','g']:
        if num < 1000.0:
            return "%3.1f%s" % (num, x)
        num /= 1000.0
    return "%3.1f%s" % (num, 't')

def generate_itemset(row_count, max_per_basket, num_freq_sets, item_count, prob_frequent):
    '''
    Generate a dataset of frequent items. These paramaters can be changed to 
    determine the type of data to generate.

    :param int row_count: The number of rows in the dataset.
    :param int max_per_basket: Maximum number of items per basket.
    :param int num_freq_sets: The number of unique frequent item sets.
    :param int item_count: The number of unique items.
    :param float prob_frequent: The probability of a basket containing a frequent itemset.
    '''
    # Generate the data
    pop_frequent = ["F"+str(n) for n in range(0,max_per_basket)]
    pop_regular = ["I"+str(n) for n in range(max_per_basket,item_count)]
    freq_itemsets = []

    # Create a filename that encodes the max_per_basket and basket_count into
    # the filename.
    filename = str(prob_frequent)+"_tsz" \
        + str(max_per_basket)+'_tct' \
         +sizeof_fmt(row_count)+'.txt'

    for i in tqdm(range(num_freq_sets),desc=f"{filename}:pass 1/2"):
        cnt = random.randint(1,max_per_basket)
        freq_itemsets.append(random.sample(pop_frequent,cnt))

    with open(filename, 'w') as f:
        for i in tqdm(range(row_count),desc=f"{filename}:pass 2/2"):
            line = []

            cnt = random.randint(1,max_per_basket)
            if random.random()<=prob_frequent:
                idx = random.randint(0,len(freq_itemsets)-1)
                for j in range(len(freq_itemsets[idx])):
                    line.append(freq_itemsets[idx][j])

            needed = max(0,cnt - len(line))
            line = line + random.sample(pop_regular,needed)

            f.write(" ".join(line)+"\n")

random.seed(1000)
ROWS = 1000000

for i in range(10,110,10):
    generate_itemset(ROWS, i, 100, 50000, 0.5)

for i in range(1,9):
    generate_itemset(ROWS, 50, 100, 50000, i/10.0)

0.8_tsz50_tct1.0m.txt:pass 2/2: 100%|██████████| 1000000/1000000 [00:19<00:00, 50642.81it/s]


In [11]:
    #Generate a dataset of frequent items. These paramaters can be changed to 
    #determine the type of data to generate.

    #:param int row_count: The number of rows in the dataset. (in this case they are 10000)
    #:param int max_per_basket: Maximum number of items per basket. (max 100 items per basket)
    #:param int num_freq_sets: The number of unique frequent item sets. (100 freq item sets)
    #:param int item_count: The number of unique items. (500 unique items)
    #:param float prob_frequent: The probability of a basket containing a frequent itemset. (50% is the probability that a basket contains a freq itemset)

0.5_tsz50_tct1.0m.txt:pass 2/2: 100%|██████████| 1000000/1000000 [00:16<00:00, 59359.52it/s]


In [4]:

size_dataset_txt = ['0.5_tsz10_tct1.0m.txt',
                    '0.5_tsz20_tct1.0m.txt',
                    '0.5_tsz30_tct1.0m.txt',
                    '0.5_tsz40_tct1.0m.txt',
                    '0.5_tsz50_tct1.0m.txt',
                    '0.5_tsz60_tct1.0m.txt',
                    '0.5_tsz70_tct1.0m.txt',
                    '0.5_tsz80_tct1.0m.txt',
                    '0.5_tsz90_tct1.0m.txt',
                    '0.5_tsz100_tct1.0m.txt'   
]

density_dataset_txt = ['0.1_tsz10_tct1.0m.txt',
                       '0.2_tsz10_tct1.0m.txt',
                       '0.3_tsz10_tct1.0m.txt',
                       '0.4_tsz10_tct1.0m.txt',
                       '0.5_tsz10_tct1.0m.txt',
                       '0.6_tsz10_tct1.0m.txt',
                       '0.7_tsz10_tct1.0m.txt',
                       '0.8_tsz10_tct1.0m.txt'    
]

In [141]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('0.5_tsz80_tct1.0m.txt')

In [142]:
# I want the dataset to be a np.array
dataset_array = np.array(dataset)

print(dataset_array[0])

# since the array contains baskets with only a string of element and not a list of elements, I need to fix this thing
new_array = []
for i in range(len(dataset_array)):
    split_elements = np.char.split(dataset_array[i].astype(str))
    ok_split_elements = (split_elements[0])
    new_array.append(ok_split_elements)

['F58 F66 F19 F7 I6602 I34535 I10500 I912 I44289 I41717 I20554 I47947 I8027 I12281 I38806 I24663 I24804 I49051 I28648 I8735 I42448 I10561 I31527 I2651 I41046 I2674 I26521 I44773 I20476 I19427 I28188 I27647 I23926 I15695 I14441 I18096 I18333 I2387 I36217 I33109 I27027 I39696 I28709 I19415 I32984 I39805 I7761 I2209 I48343 I2792 I37557 I4646 I2886 I27492 I43364 I28704 I49617 I28934 I6998 I16290 I47993 I13780 I46717 I35138 I22820 I46419 I6972 I21899 I8578 I45488']


In [143]:
# now it works
#new_array[1]

In [144]:
#new_array[0]

In [145]:
#len(new_array[0])

#apriori



In [146]:
#!pip install apyori

In [147]:
'''from apyori import apriori
import time

# Example dataset
dataset = new_array

# Minimum support threshold
min_support = 0.2

start = time.time()
# Apply the Apriori algorithm
results = list(apriori(dataset, min_support=min_support))

# Print frequent itemsets and their support
for itemset in results:
    items = list(itemset.items)
    support = itemset.support
    print(items, support)

end = time.time()

print('Time required:', end - start)'''


"from apyori import apriori\nimport time\n\n# Example dataset\ndataset = new_array\n\n# Minimum support threshold\nmin_support = 0.2\n\nstart = time.time()\n# Apply the Apriori algorithm\nresults = list(apriori(dataset, min_support=min_support))\n\n# Print frequent itemsets and their support\nfor itemset in results:\n    items = list(itemset.items)\n    support = itemset.support\n    print(items, support)\n\nend = time.time()\n\nprint('Time required:', end - start)"

#fp-growth
https://github.com/JackHCC/Apriori-and-FP_Growth/blob/master/FP_growth.py


In [148]:
#fp-growth

# fp-growth

from collections import defaultdict, namedtuple
import time

def find_frequent_itemsets(transactions, minimum_support, include_support=False):
    # Function to find frequent itemsets in the transactions dataset

    items = defaultdict(lambda: 0)

    for transaction in transactions:
        for item in transaction:
            items[item] += 1

    # Filtering frequent 1-itemsets
    items = dict((item, support) for item, support in items.items()
        if support >= minimum_support)

    def clean_transaction(transaction):
        transaction = filter(lambda v: v in items, transaction)
        transaction_list = list(transaction)
        transaction_list.sort(key=lambda v: items[v], reverse=True)
        return transaction_list

    # Constructing the FP-tree
    master = FPTree()
    for transaction in map(clean_transaction, transactions):
        master.add(transaction)

    def find_with_suffix(tree, suffix):
        for item, nodes in tree.items():
            support = sum(n.count for n in nodes)
            if support >= minimum_support and item not in suffix:
                found_set = [item] + suffix
                yield (found_set, support) if include_support else found_set

                cond_tree = conditional_tree_from_paths(tree.prefix_paths(item))
                for s in find_with_suffix(cond_tree, found_set):
                    yield s

    for itemset in find_with_suffix(master, []):
        yield itemset

class FPTree(object):
    # FP-tree data structure

    Route = namedtuple('Route', 'head tail')

    def __init__(self):
        # Initialize the root node and routes
        self._root = FPNode(self, None, None)
        self._routes = {}

    @property
    def root(self):
        # Create the root node
        return self._root

    def add(self, transaction):
        # Add a transaction to the tree
        point = self._root

        for item in transaction:
            next_point = point.search(item)
            if next_point:
                # Current node already exists
                next_point.increment()
            else:
                # Create a new node
                next_point = FPNode(self, item)
                point.add(next_point)

                # Update the routes
                self._update_route(next_point)

            point = next_point

    def _update_route(self, point):
        assert self is point.tree

        try:
            route = self._routes[point.item]
            route[1].neighbor = point  # route[1] is the tail
            self._routes[point.item] = self.Route(route[0], point)
        except KeyError:
            # Start a new node
            self._routes[point.item] = self.Route(point, point)

    def items(self):
        # Iterate over the items in the tree
        for item in self._routes.keys():
            yield (item, self.nodes(item))

    def nodes(self, item):
        # Iterate over the nodes associated with an item
        try:
            node = self._routes[item][0]
        except KeyError:
            return

        while node:
            yield node
            node = node.neighbor

    def prefix_paths(self, item):
        # Generate prefix paths for an item
        def collect_path(node):
            path = []
            while node and not node.root:
                path.append(node)
                node = node.parent
            path.reverse()
            return path

        return (collect_path(node) for node in self.nodes(item))

    def inspect(self):
        # Print the tree structure
        self.root.inspect(1)

        for item, nodes in self.items():
            for node in nodes:
                print('    %r' % node)

def conditional_tree_from_paths(paths):
    # Construct a conditional tree from paths
    tree = FPTree()
    condition_item = None
    items = set()

    for path in paths:
        if condition_item is None:
            condition_item = path[-1].item

        point = tree.root
        for node in path:
            next_point = point.search(node.item)
            if not next_point:
                # Add a new node to the tree
                items.add(node.item)
                count = node.count if node.item == condition_item else 0
                next_point = FPNode(tree, node.item, count)
                point.add(next_point)
                tree._update_route(next_point)
            point = next_point

    assert condition_item is not None

    # Calculate the counts for nodes
    for path in tree.prefix_paths(condition_item):
        count = path[-1].count
        for node in reversed(path[:-1]):
            node._count += count

    return tree

class FPNode(object):
    # FP-tree node

    def __init__(self, tree, item, count=1):
        self._tree = tree
        self._item = item
        self._count = count
        self._parent = None
        self._children = {}
        self._neighbor = None

    def add(self, child):
        # Add a child node

        if not isinstance(child, FPNode):
            raise TypeError("Can only add other FPNodes as children")

        if not child.item in self._children:
            self._children[child.item] = child
            child.parent = self

    def search(self, item):
        # Search for a child node with the given item
        try:
            return self._children[item]
        except KeyError:
            return None

    def __contains__(self, item):
        return item in self._children

    @property
    def tree(self):
        return self._tree

    @property
    def item(self):
        return self._item

    @property
    def count(self):
        return self._count

    def increment(self):
        if self._count is None:
            raise ValueError("Root nodes have no associated count.")
        self._count += 1

    @property
    def root(self):
        return self._item is None and self._count is None

    @property
    def leaf(self):
        return len(self._children) == 0

    @property
    def parent(self):
        return self._parent

    @parent.setter
    def parent(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a parent.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a parent from another tree.")
        self._parent = value

    @property
    def neighbor(self):
        return self._neighbor

    @neighbor.setter
    def neighbor(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a neighbor.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a neighbor from another tree.")
        self._neighbor = value

    @property
    def children(self):
        return tuple(self._children.values())

    def inspect(self, depth=0):
        for child in self.children:
            child.inspect(depth + 1)

    def __repr__(self):
        if self.root:
            return "<%s (root)>" % type(self).__name__
        return "<%s %r (%r)>" % (type(self).__name__, self.item, self.count)


In [149]:
# Dataset
dataset = new_array
# Main function
if __name__ == '__main__':
    # Call find_frequent_itemsets() to generate frequent itemsets
    start = time.time()

    total_baskets = len(dataset)
    min_support = 0.2 * total_baskets

    frequent_itemsets = find_frequent_itemsets(dataset, minimum_support=min_support, include_support=True)

    result = []
    for itemset, support in frequent_itemsets:
        result.append((itemset, support))

    result = sorted(result, key=lambda i: i[0])
    for itemset, support in result:
        print(str(itemset) + ' ' + str(support/total_baskets))

    end = time.time()
    print('Execution time:', str(end - start))


['F0'] 0.24932624932624933
['F1'] 0.2605192605192605
['F10'] 0.2653902653902654
['F10', 'F52'] 0.20534920534920534
['F11'] 0.2502182502182502
['F12'] 0.24957924957924957
['F13'] 0.24498024498024498
['F14'] 0.23017223017223018
['F15'] 0.24503524503524504
['F16'] 0.28527128527128526
['F16', 'F1'] 0.20024820024820025
['F16', 'F18'] 0.21034121034121034
['F16', 'F22'] 0.2101002101002101
['F16', 'F28'] 0.20018120018120017
['F16', 'F29'] 0.21498521498521497
['F16', 'F33'] 0.20977420977420977
['F16', 'F36'] 0.20017320017320017
['F16', 'F51'] 0.20006020006020006
['F16', 'F54'] 0.20494620494620494
['F16', 'F56'] 0.20031320031320032
['F16', 'F62'] 0.20519220519220518
['F16', 'F65'] 0.20975620975620976
['F16', 'F68'] 0.2001882001882002
['F16', 'F69'] 0.20474520474520475
['F16', 'F7'] 0.20532120532120532
['F16', 'F8'] 0.2002972002972003
['F17'] 0.23516823516823518
['F18'] 0.26526026526026525
['F19'] 0.2798332798332798
['F19', 'F10'] 0.21003321003321004
['F19', 'F20'] 0.20011020011020011
['F19', 'F3

# eclat
https://github.com/jagdeep227/Frequent_itemset_mining/blob/main/Eclat_g.py


In [150]:
# eclat
#eclat
import time

FreqItems = dict()
support = dict()


def eclat(prefix, items, dict_id ,minsup,cnt):
    
    while items:
        i ,itids = items.pop()
        isupp = len(itids)
        if isupp >= minsup*cnt:
            print(prefix + [i], isupp/len(dataset))
            FreqItems[frozenset(prefix + [i])] = isupp
            suffix = []
            for j, ojtids in items:
                jtids = itids & ojtids
                if len(jtids ) >= minsup*cnt:
                    suffix.append((j, jtids))
            dict_id += 1
            eclat(prefix +[i], sorted(suffix, key=lambda item: len(item[1]), reverse=True), dict_id ,minsup,cnt)

In [151]:
items = {}

for transaction_id, transaction in enumerate(dataset):
    for item in transaction:
        if item in items:
            items[item].add(transaction_id)
        else:
            items[item] = {transaction_id}

    # Convert the dictionary items into a list of tuples
items = [(item, itids) for item, itids in items.items()]

In [152]:
# Dataset
dataset = new_array
# Main function
if __name__ == '__main__':
    # Call find_frequent_itemsets() to generate frequent itemsets
    start = time.time()

    cnt = len(dataset)
    minsup = 0.2
    # Function call to start ECLAT algorithm
    eclat([], sorted(items, key=lambda item: len(item[1]), reverse=True), 1, minsup, cnt)

    result = []
    for itemset, support in frequent_itemsets:
        result.append((itemset, support))

    result

    end = time.time()
    print('Execution time:', str(end - start))

['F71'] 0.21532621532621532
['F57'] 0.21536921536921536
['F77'] 0.21540321540321541
['F67'] 0.2250932250932251
['F48'] 0.22978022978022977
['F41'] 0.2299132299132299
['F47'] 0.22995722995722995
['F39'] 0.23013823013823015
['F14'] 0.23017223017223018
['F76'] 0.23504323504323504
['F38'] 0.23506923506923508
['F38', 'F63'] 0.2000022000022
['F32'] 0.2350922350922351
['F2'] 0.23515123515123515
['F2', 'F29'] 0.20025820025820026
['F17'] 0.23516823516823518
['F73'] 0.23521723521723523
['F46'] 0.23952623952623953
['F46', 'F75'] 0.20433520433520433
['F50'] 0.2397002397002397
['F3'] 0.23973423973423974
['F72'] 0.2401022401022401
['F35'] 0.2401062401062401
['F64'] 0.24014624014624014
['F66'] 0.2402822402822403
['F31'] 0.24424824424824426
['F25'] 0.24457624457624458
['F45'] 0.24479524479524478
['F13'] 0.24498024498024498
['F15'] 0.24503524503524504
['F15', 'F33'] 0.20993120993120992
['F5'] 0.24525224525224526
['F5', 'F56'] 0.20527820527820528
['F5', 'F63'] 0.20528520528520527
['F68'] 0.2453482453482